### Load in the training and test data

In [46]:
import pandas as pd
import numpy as np

# Use read_excel and use participant_id as the index
train_cat = pd.read_excel("../../widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_func = pd.read_csv("../../widsdatathon2025/TRAIN_NEW/"
                         "TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_quant = pd.read_excel("../../widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_soln = pd.read_excel("../../widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

# TEST
test_cat = pd.read_excel("../../widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_func = pd.read_csv("../../widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")
test_quant = pd.read_excel("../../widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
solutions_path = "../../widsdatathon2025/TEST/TEST_SOLUTIONS.xlsx"

In [47]:
# Set index for merging
train_cat.set_index("participant_id", inplace=True)
train_quant.set_index("participant_id", inplace=True)
train_func.set_index("participant_id", inplace=True)
train_soln.set_index("participant_id", inplace=True)

# TEST
test_cat.set_index("participant_id", inplace=True)
test_quant.set_index("participant_id", inplace=True)
test_func.set_index("participant_id", inplace=True)

In [48]:
train_func

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,0.470063,...,0.224985,0.397448,0.422966,0.184642,0.305549,0.420349,0.016328,0.561864,0.471170,0.365221
WHWymJu6zNZi,0.614765,0.577255,0.496127,0.496606,0.404686,0.439724,0.122590,-0.085452,0.120673,0.276350,...,0.217546,-0.014549,0.000440,-0.096451,0.454501,0.343916,0.167313,0.607656,0.550623,0.503176
4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,0.378557,...,0.342487,-0.021141,-0.037836,0.075069,0.412712,0.292708,0.391005,0.461544,0.508912,0.624232
obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,0.259596,...,0.103562,-0.178313,0.210983,-0.018666,0.436313,0.592982,0.216205,0.341272,0.440313,0.558193
s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,0.331445,...,-0.164956,0.007064,-0.120904,-0.488095,0.493575,-0.215361,0.210685,0.055850,0.119065,0.108273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9gpepMI9sj5q,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,0.340388,...,0.127097,0.129787,0.298987,0.114577,0.534109,0.118893,0.181292,0.181055,0.238357,0.577009
FIDen5rdMc0v,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,0.434427,...,0.482214,-0.159587,-0.162498,-0.093249,0.309914,0.143818,0.218337,0.389331,0.328741,0.238443
dlsMC4TXL4e8,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,-0.357953,...,-0.379304,0.126976,0.193695,0.214483,0.429836,0.302141,0.104774,0.856375,0.303248,0.363639


## PCA for the functional connectome matrix to reduce dimension

In [49]:
from sklearn.decomposition import PCA

# Select only fMRI columns
fmri_data = train_func.drop(columns=['participant_id'], errors='ignore')
fmri_data_test = test_func.drop(columns=['participant_id'], errors='ignore')

# Apply PCA to reduce to top N components (e.g. 50)
pca = PCA(n_components=50)
fmri_pca = pca.fit_transform(fmri_data)
fmri_pca_test = pca.fit_transform(fmri_data_test)

train_func_pca = pd.DataFrame(fmri_pca, index=train_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
test_func_pca = pd.DataFrame(fmri_pca_test, index=test_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
train_func_pca.head()

,fmri_pca_1,fmri_pca_2,fmri_pca_3,fmri_pca_4,fmri_pca_5,fmri_pca_6,fmri_pca_7,fmri_pca_8,fmri_pca_9,fmri_pca_10,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,6.322800,-0.365680,-4.918161,-3.823234,-5.706923,-0.875163,1.567536,-1.347507,-1.717449,1.626939,...,0.434735,-0.387117,0.531641,-0.462497,-0.270997,4.458481,2.125456,0.415437,-0.327486,0.903944
WHWymJu6zNZi,5.468294,-3.985252,1.542399,0.104719,-0.529029,-2.951971,0.786358,0.794932,-2.771925,-2.408927,...,1.326487,0.868321,-1.112521,0.660175,-0.903732,0.041896,-0.479450,0.350675,-0.185789,-0.228479
4PAQp1M6EyAo,0.447190,0.733000,3.051331,1.763904,1.932821,6.451160,-3.356872,1.946050,-1.165671,-2.876979,...,-2.188954,-0.707309,2.339618,0.554921,-1.037176,1.372949,0.544717,1.520982,-1.512680,0.261823
obEacy4Of68I,-9.149799,-2.044626,0.519359,-2.703924,-4.444127,2.916929,1.036388,0.080338,-0.148184,0.384193,...,0.250241,-1.731520,0.123517,-0.680417,-1.345387,0.029469,0.431224,-0.554442,-1.167923,1.656055
s7WzzDcmDOhF,0.812814,-1.933265,1.834524,0.383159,-0.512341,2.972217,3.151068,3.419854,0.283657,-2.997717,...,2.333038,-1.649248,-2.849112,-0.690900,0.126466,-1.022288,-0.442141,2.301413,0.150213,-1.417518


## Interpolate missing values with mean for quant and cat dfs

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

def impute_with_regression(df, target_col):
    """Impute missing values in target_col of df using linear regression."""
    df = df.copy()
    train_df = df.dropna(subset=[target_col])
    if train_df.empty:
        return df  # skip if no data to train on
    X_train = train_df.drop(columns=[target_col])
    y_train = train_df[target_col]
    X_train_imp = SimpleImputer(strategy='mean').fit_transform(X_train)
    reg = LinearRegression().fit(X_train_imp, y_train)

    missing_mask = df[target_col].isna()
    X_missing = df.loc[missing_mask].drop(columns=[target_col])
    X_missing_imp = SimpleImputer(strategy='mean').fit_transform(X_missing)
    df.loc[missing_mask, target_col] = reg.predict(X_missing_imp)
    return df

# Function to process any dataframe (quant or cat)
def process_df(df, name='df'):
    missing = df.isnull().sum()
    for col in missing[missing > 0].index:
        if missing[col] < 50:
            df[col] = df[col].fillna(df[col].mean())
        else:
            print(f"Using regression to impute '{col}' in {name}")
            df = impute_with_regression(df, col)
    return df

# Apply to all four
train_quant = process_df(train_quant, 'train_quant')
train_cat = process_df(train_cat, 'train_cat')
test_quant = process_df(test_quant, 'test_quant')
test_cat = process_df(test_cat, 'test_cat')

Using regression to impute 'MRI_Track_Age_at_Scan' in train_quant
Using regression to impute 'PreInt_Demos_Fam_Child_Race' in train_cat
Using regression to impute 'Barratt_Barratt_P2_Edu' in train_cat
Using regression to impute 'Barratt_Barratt_P2_Occ' in train_cat


In [51]:
# Check missing values *before* merging
missing = train_quant.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])

Missing values in train_quant:
 Series([], dtype: int64)


In [52]:
train_quant.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,14.278626
04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574


## One-Hot-Encoding on train_cat and test_cat to encode categorical variables with no hierarchy 
Turn the numbers into their labels using the values provided in the data dictionary.
Use pandas to perform one-hot encoding so that the models can interpret the binary values instead of arbitrary numbers.

In [53]:
# Categorical Encodings
Basic_Demos_Study_Site = {
    1: "Staten Island",
    2: "MRV",
    3: "Midtown",
    4: "Harlem",
    5: "SI RUMC",
}
PreInt_Demos_Fam_Child_Ethnicity = {
    0: "Not Hispanic or Latino",
    1: "Hispanic or Latino",
    2: "Decline to specify",
    3: "Unknown"
}
PreInt_Demos_Fam_Child_Race = {
  0: "White/Caucasian",
  1: "Black/African American",
  2: "Hispanic",
  3: "Asian",
  4: "Indian",
  5: "Native American Indian",
  6: "American Indian/Alaskan Native",
  7: "Native Hawaiian/Other Pacific Islander",
  8: "Two or more races",
  9: "Other race",
  10: "Unknown",
  11: "Chose not to specify"}
MRI_Track_Scan_Location = {
    1: "Staten Island",
    2: "RUBIC", 
    3: "CBIC",
    4: "CUNY"
}
Barratt_Barratt_P1_Edu = {
    3: "Less than 7th grade",
    6: "Junior high/Middle school (9th grade)",
    9: "Partial high school (10th or 11th grade)",
    12: "High school graduate",
    15: "Partial college (at least one year)",
    18: "College education",
    21: "Graduate degree"
}
Barratt_Barratt_P1_Occ = {
    0: "Homemaker, stay at home parent.",
    5: "Day laborer, janitor, house cleaner, farm worker, food counter sales, food preparation worker, busboy.",
    10: "Garbage collector, short-order cook, cab driver, shoe sales, assembly line workers, masons, baggage porter.",
    15: "Painter, skilled construction trade, sales clerk, truck driver, cook, sales counter or general office clerk.",
    20: "Automobile mechanic, typist, locksmith, farmer, carpenter, receptionist, construction laborer, hairdresser.",
    25: "Machinist, musician, bookkeeper, secretary, insurance sales, cabinet maker, personnel specialist, welder.",
    30: "Supervisor, librarian, aircraft mechanic, artist and artisan, electrician, administrator, military enlisted personnel, buyer.",
    35: "Nurse, skilled technician, medical technician, counselor, manager, police and fire personnel, financial manager, physical, occupational, speech therapist.",
    40: "Mechanical, nuclear, and electrical engineer, educational administrator, veterinarian, military officer, elementary, high school and special education teacher.",
    45: "Physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant."
}
Barratt_Barratt_P2_Edu = {
    3: "Less than 7th grade",
    6: "Junior high/Middle school (9th grade)",
    9: "Partial high school (10th or 11th grade)",
    12: "High school graduate",
    15: "Partial college (at least one year)",
    18: "College education",
    21: "Graduate degree"
}
Barratt_Barratt_P2_Occ = {
    0: "Homemaker, stay at home parent.",
    5: "Day laborer, janitor, house cleaner, farm worker, food counter sales, food preparation worker, busboy.",
    10: "Garbage collector, short-order cook, cab driver, shoe sales, assembly line workers, masons, baggage porter.",
    15: "Painter, skilled construction trade, sales clerk, truck driver, cook, sales counter or general office clerk.",
    20: "Automobile mechanic, typist, locksmith, farmer, carpenter, receptionist, construction laborer, hairdresser.",
    25: "Machinist, musician, bookkeeper, secretary, insurance sales, cabinet maker, personnel specialist, welder.",
    30: "Supervisor, librarian, aircraft mechanic, artist and artisan, electrician, administrator, military enlisted personnel, buyer.",
    35: "Nurse, skilled technician, medical technician, counselor, manager, police and fire personnel, financial manager, physical, occupational, speech therapist.",
    40: "Mechanical, nuclear, and electrical engineer, educational administrator, veterinarian, military officer, elementary, high school and special education teacher.",
    45: "Physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant.",
}

# Apply label replacements using the dictionaries
train_cat["Basic_Demos_Study_Site"] = train_cat["Basic_Demos_Study_Site"].replace(Basic_Demos_Study_Site)
train_cat["PreInt_Demos_Fam_Child_Ethnicity"] = train_cat["PreInt_Demos_Fam_Child_Ethnicity"].replace(PreInt_Demos_Fam_Child_Ethnicity)
train_cat["PreInt_Demos_Fam_Child_Race"] = train_cat["PreInt_Demos_Fam_Child_Race"].replace(PreInt_Demos_Fam_Child_Race)
train_cat["MRI_Track_Scan_Location"] = train_cat["MRI_Track_Scan_Location"].replace(MRI_Track_Scan_Location)
train_cat["Barratt_Barratt_P1_Edu"] = train_cat["Barratt_Barratt_P1_Edu"].replace(Barratt_Barratt_P1_Edu)
train_cat["Barratt_Barratt_P1_Occ"] = train_cat["Barratt_Barratt_P1_Occ"].replace(Barratt_Barratt_P1_Occ)
train_cat["Barratt_Barratt_P2_Edu"] = train_cat["Barratt_Barratt_P2_Edu"].replace(Barratt_Barratt_P2_Edu)
train_cat["Barratt_Barratt_P2_Occ"] = train_cat["Barratt_Barratt_P2_Occ"].replace(Barratt_Barratt_P2_Occ)

# One-hot encode all columns except 'participant_id' and 'Basic_Demos_Enroll_Year' for train
train_cat_encoded = pd.get_dummies(
    train_cat,
    columns=[
        "Basic_Demos_Study_Site",
        "PreInt_Demos_Fam_Child_Ethnicity",
        "PreInt_Demos_Fam_Child_Race",
        "MRI_Track_Scan_Location",
        "Barratt_Barratt_P1_Edu",
        "Barratt_Barratt_P1_Occ",
        "Barratt_Barratt_P2_Edu",
        "Barratt_Barratt_P2_Occ",
    ],
    drop_first=False,  # Set to True if you want to drop the first category to avoid multicollinearity
)

train_cat_encoded = train_cat_encoded.astype(int)

# Apply label replacements using the dictionaries for test set
test_cat["Basic_Demos_Study_Site"] = test_cat["Basic_Demos_Study_Site"].replace(Basic_Demos_Study_Site)
test_cat["PreInt_Demos_Fam_Child_Ethnicity"] = test_cat["PreInt_Demos_Fam_Child_Ethnicity"].replace(PreInt_Demos_Fam_Child_Ethnicity)
test_cat["PreInt_Demos_Fam_Child_Race"] = test_cat["PreInt_Demos_Fam_Child_Race"].replace(PreInt_Demos_Fam_Child_Race)
test_cat["MRI_Track_Scan_Location"] = test_cat["MRI_Track_Scan_Location"].replace(MRI_Track_Scan_Location)
test_cat["Barratt_Barratt_P1_Edu"] = test_cat["Barratt_Barratt_P1_Edu"].replace(Barratt_Barratt_P1_Edu)
test_cat["Barratt_Barratt_P1_Occ"] = test_cat["Barratt_Barratt_P1_Occ"].replace(Barratt_Barratt_P1_Occ)
test_cat["Barratt_Barratt_P2_Edu"] = test_cat["Barratt_Barratt_P2_Edu"].replace(Barratt_Barratt_P2_Edu)
test_cat["Barratt_Barratt_P2_Occ"] = test_cat["Barratt_Barratt_P2_Occ"].replace(Barratt_Barratt_P2_Occ)

# One-hot encode all columns for the test set (with the same categories as train)
test_cat_encoded = pd.get_dummies(
    test_cat,
    columns=[
        "Basic_Demos_Study_Site",
        "PreInt_Demos_Fam_Child_Ethnicity",
        "PreInt_Demos_Fam_Child_Race",
        "MRI_Track_Scan_Location",
        "Barratt_Barratt_P1_Edu",
        "Barratt_Barratt_P1_Occ",
        "Barratt_Barratt_P2_Edu",
        "Barratt_Barratt_P2_Occ",
    ],
    drop_first=False,
)

test_cat_encoded = test_cat_encoded.astype(int)

test_cat_encoded = test_cat_encoded.reindex(columns=train_cat_encoded.columns, fill_value=0)

train_cat_encoded.head()
#test_cat_encoded.head()

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site_Harlem,Basic_Demos_Study_Site_MRV,Basic_Demos_Study_Site_Midtown,Basic_Demos_Study_Site_Staten Island,PreInt_Demos_Fam_Child_Ethnicity_0.4358974358974359,PreInt_Demos_Fam_Child_Ethnicity_Decline to specify,PreInt_Demos_Fam_Child_Ethnicity_Hispanic or Latino,PreInt_Demos_Fam_Child_Ethnicity_Not Hispanic or Latino,PreInt_Demos_Fam_Child_Ethnicity_Unknown,...,"Barratt_Barratt_P2_Occ_Automobile mechanic, typist, locksmith, farmer, carpenter, receptionist, construction laborer, hairdresser.","Barratt_Barratt_P2_Occ_Day laborer, janitor, house cleaner, farm worker, food counter sales, food preparation worker, busboy.","Barratt_Barratt_P2_Occ_Garbage collector, short-order cook, cab driver, shoe sales, assembly line workers, masons, baggage porter.","Barratt_Barratt_P2_Occ_Homemaker, stay at home parent.","Barratt_Barratt_P2_Occ_Machinist, musician, bookkeeper, secretary, insurance sales, cabinet maker, personnel specialist, welder.","Barratt_Barratt_P2_Occ_Mechanical, nuclear, and electrical engineer, educational administrator, veterinarian, military officer, elementary, high school and special education teacher.","Barratt_Barratt_P2_Occ_Nurse, skilled technician, medical technician, counselor, manager, police and fire personnel, financial manager, physical, occupational, speech therapist.","Barratt_Barratt_P2_Occ_Painter, skilled construction trade, sales clerk, truck driver, cook, sales counter or general office clerk.","Barratt_Barratt_P2_Occ_Physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant.","Barratt_Barratt_P2_Occ_Supervisor, librarian, aircraft mechanic, artist and artisan, electrician, administrator, military enlisted personnel, buyer."
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,2019,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
00fV0OyyoLfw,2017,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
04X1eiS79T4B,2017,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
05ocQutkURd6,2018,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
06YUNBA9ZRLq,2018,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Feature Selection for train_quant using K clustering

In [54]:
# make participant_id a column
train_quant = train_quant.reset_index()
train_cat = train_cat_encoded.reset_index()
train_soln = train_soln.reset_index()
test_quant = test_quant.reset_index()
test_cat = test_cat_encoded.reset_index()

# Merge everything on participant_id
X_full_train = train_quant.merge(train_cat, on='participant_id').merge(train_func_pca, on='participant_id')
X_full_test = test_quant.merge(test_cat, on='participant_id').merge(test_func_pca, on='participant_id')

# Target labels for model training
y = train_soln[['participant_id', 'ADHD_Outcome', 'Sex_F']]

# Align rows across X and y
X_full_train = X_full_train.merge(y, on='participant_id')
X_full_train = X_full_train.drop(columns=['ADHD_Outcome', 'Sex_F'])
X_full_train.set_index("participant_id", inplace=True)
X_full_test.set_index("participant_id", inplace=True)
X_full_test.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,60.03,14.0,5.0,16.0,41.0,19.0,11.0,26.0,2.0,12.0,...,-1.742198,-1.104064,0.493018,-0.476232,2.631934,0.537296,1.785159,1.578776,0.457824,-0.405805
vhGrzmvA3Hjq,86.71,12.0,3.0,13.0,43.0,18.0,15.0,28.0,2.0,16.0,...,-1.084569,-2.001920,-0.632775,2.851724,-0.586700,-3.291825,-0.035094,1.291278,1.895738,-0.000252
ULliyEXjy4OV,26.68,13.0,3.0,14.0,36.0,16.0,14.0,25.0,1.0,7.0,...,0.495617,0.194214,-1.124200,0.301353,3.017943,1.211216,4.116044,-0.649670,-0.530827,0.357612
LZfeAb1xMtql,93.38,13.0,3.0,19.0,41.0,17.0,18.0,27.0,4.0,15.0,...,1.671720,-0.724728,0.528166,-0.884385,0.080298,0.295043,2.343871,0.212618,1.100562,0.281359
EnFOUv0YK1RG,-93.38,14.0,3.0,13.0,42.0,19.0,16.0,28.0,2.0,18.0,...,-0.550988,0.403766,-0.758231,2.247869,-0.461880,-2.343593,0.543796,-2.583071,0.462248,-1.472230


In [55]:
X_full_train.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,...,0.074771,-0.867133,-1.320011,-0.621475,-0.889508,1.811582,-0.807552,-0.879140,4.197603,-3.906602
00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,...,-0.274862,-0.037864,0.193410,-1.059520,0.250155,-0.769743,3.121714,-0.859271,2.146147,-0.817614
04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,...,3.698518,-2.243388,-1.860986,0.599041,-2.419437,-0.638312,0.255009,-0.072727,-2.781938,-2.517083
05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,...,1.786929,0.233774,2.180655,-1.409129,-4.225646,0.688282,-1.482971,-2.149111,-2.142751,0.844207
06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,...,1.854073,1.644142,0.751862,0.305654,-1.857752,-1.340609,1.679403,-1.592114,0.965482,0.927551


In [56]:
# Combine with targets temporarily
X_full_train = X_full_train.reset_index()
X_with_targets = X_full_train.merge(train_soln[['participant_id', 'ADHD_Outcome', 'Sex_F']], on='participant_id')

# Compute correlations
corrs = X_with_targets.drop(columns='participant_id').corr()

# Grab correlation with targets
adhd_corr = corrs['ADHD_Outcome'].abs().sort_values(ascending=False)
sex_corr = corrs['Sex_F'].abs().sort_values(ascending=False)

# Keep top N or those above a threshold
threshold = 0.05  # adjust this as needed
top_adhd_feats = adhd_corr[adhd_corr > threshold].index.tolist()
top_sex_feats = sex_corr[sex_corr > threshold].index.tolist()

# Union of both sets, drop targets themselves if included
quant_selected = list(set(top_adhd_feats + top_sex_feats) - {'ADHD_Outcome', 'Sex_F'})

quant_selected

['fmri_pca_13',
 'fmri_pca_2',
 'ColorVision_CV_Score',
 'fmri_pca_41',
 'fmri_pca_46',
 'MRI_Track_Scan_Location_CUNY',
 'MRI_Track_Scan_Location_CBIC',
 'APQ_P_APQ_P_CP',
 'SDQ_SDQ_Prosocial',
 'fmri_pca_14',
 'fmri_pca_26',
 'fmri_pca_1',
 'Barratt_Barratt_P1_Edu_Partial high school (10th or 11th grade)',
 'SDQ_SDQ_Hyperactivity',
 'fmri_pca_38',
 'fmri_pca_7',
 'MRI_Track_Scan_Location_Staten Island',
 'APQ_P_APQ_P_ID',
 'fmri_pca_40',
 'fmri_pca_39',
 'Barratt_Barratt_P2_Edu_Less than 7th grade',
 'fmri_pca_29',
 'fmri_pca_44',
 'PreInt_Demos_Fam_Child_Ethnicity_0.4358974358974359',
 'fmri_pca_30',
 'PreInt_Demos_Fam_Child_Race_Black/African American',
 'SDQ_SDQ_Internalizing',
 'APQ_P_APQ_P_OPD',
 'fmri_pca_24',
 'Basic_Demos_Enroll_Year',
 'SDQ_SDQ_Generating_Impact',
 'SDQ_SDQ_Difficulties_Total',
 'SDQ_SDQ_Peer_Problems',
 'Basic_Demos_Study_Site_Staten Island',
 'Basic_Demos_Study_Site_Midtown',
 'fmri_pca_15',
 'fmri_pca_50',
 'fmri_pca_21',
 'APQ_P_APQ_P_INV',
 'fmri_pca_31

In [57]:
# Merge everything on participant_id
X_full_test = X_full_test.reset_index()
X_select_train = X_full_train[['participant_id'] + quant_selected]
X_select_test = X_full_test[['participant_id'] + quant_selected]

# Target labels
y = train_soln[['participant_id', 'ADHD_Outcome', 'Sex_F']]

X_select_train.head()

,participant_id,fmri_pca_13,fmri_pca_2,ColorVision_CV_Score,fmri_pca_41,fmri_pca_46,MRI_Track_Scan_Location_CUNY,MRI_Track_Scan_Location_CBIC,APQ_P_APQ_P_CP,SDQ_SDQ_Prosocial,...,fmri_pca_31,fmri_pca_33,SDQ_SDQ_Emotional_Problems,fmri_pca_32,APQ_P_APQ_P_PP,Barratt_Barratt_P1_Edu_Graduate degree,"Barratt_Barratt_P2_Occ_Homemaker, stay at home parent.",fmri_pca_22,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Externalizing
0,00aIpNTbG5uh,0.968322,-5.068500,13.0,0.074771,1.811582,0,1,3.0,9.0,...,1.192742,0.227624,4.0,6.151391,27.0,1,0,-1.761258,3.0,11.0
1,00fV0OyyoLfw,0.667914,-3.920926,14.0,-0.274862,-0.769743,0,0,3.0,8.0,...,0.589498,2.616910,4.0,0.394052,30.0,1,0,0.255715,5.0,13.0
2,04X1eiS79T4B,3.275997,-5.386339,14.0,3.698518,-0.638312,0,0,3.0,7.0,...,1.588275,-0.341572,7.0,-2.443605,28.0,0,0,1.735268,3.0,10.0
3,05ocQutkURd6,3.960646,-6.756400,14.0,1.786929,0.688282,0,0,3.0,6.0,...,0.168934,-1.882832,0.0,-2.074844,28.0,0,1,-3.901227,0.0,3.0
4,06YUNBA9ZRLq,-0.961907,-4.088958,14.0,1.854073,-1.340609,0,0,8.0,4.0,...,0.037696,-0.357489,7.0,-0.201573,24.0,0,0,-4.191496,6.0,15.0


In [58]:
X_select_test.head()

,participant_id,fmri_pca_13,fmri_pca_2,ColorVision_CV_Score,fmri_pca_41,fmri_pca_46,MRI_Track_Scan_Location_CUNY,MRI_Track_Scan_Location_CBIC,APQ_P_APQ_P_CP,SDQ_SDQ_Prosocial,...,fmri_pca_31,fmri_pca_33,SDQ_SDQ_Emotional_Problems,fmri_pca_32,APQ_P_APQ_P_PP,Barratt_Barratt_P1_Edu_Graduate degree,"Barratt_Barratt_P2_Occ_Homemaker, stay at home parent.",fmri_pca_22,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Externalizing
0,Cfwaf5FX7jWK,-1.885215,0.798518,14.0,-1.742198,0.537296,1,0,5.0,8.0,...,0.793547,-0.849166,3.0,0.227520,26.0,1,0,1.479896,2.0,9.0
1,vhGrzmvA3Hjq,3.968658,-2.606679,12.0,-1.084569,-3.291825,1,0,3.0,9.0,...,1.124243,2.003162,8.0,1.661870,28.0,1,0,-1.941712,2.0,5.0
2,ULliyEXjy4OV,-1.428745,0.156742,13.0,0.495617,1.211216,1,0,3.0,9.0,...,1.016285,0.301984,1.0,2.472538,25.0,1,0,-2.552891,1.0,6.0
3,LZfeAb1xMtql,-0.860696,-2.720716,13.0,1.671720,0.295043,0,1,3.0,6.0,...,0.764329,0.126045,4.0,-0.856699,27.0,1,0,-1.041396,4.0,10.0
4,EnFOUv0YK1RG,-3.413086,-1.047284,14.0,-0.550988,-2.343593,1,0,3.0,10.0,...,-2.821317,-0.912005,6.0,-2.390853,28.0,0,0,-2.152863,2.0,12.0


In [64]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

# Standardize
X_select_train = X_select_train.set_index('participant_id')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_select_train)

# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y[['ADHD_Outcome', 'Sex_F']].values, dtype=torch.float32)

# Stratified train/test split
stratify_labels = y['ADHD_Outcome'].astype(str) + y['Sex_F'].astype(str)
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=stratify_labels)

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

# Model
class BetterMultiOutputNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.head_adhd = nn.Linear(128, 1)
        self.head_sex = nn.Linear(128, 1)

    def forward(self, x):
        shared_out = self.shared(x)
        return self.head_adhd(shared_out), self.head_sex(shared_out)

# Custom ADHD loss
class WeightedADHDLoss(nn.Module):
    def __init__(self, weight_female_adhd=2):
        super().__init__()
        self.weight_female_adhd = weight_female_adhd
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, adhd_preds, adhd_targets, sex_targets):
        loss = self.bce(adhd_preds, adhd_targets)
        weight = torch.ones_like(loss)
        weight[(adhd_targets == 1) & (sex_targets == 1)] = self.weight_female_adhd
        return (weight * loss).mean()

# Initialize model and training setup
model = BetterMultiOutputNN(X_select_train.shape[1])
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

adhd_criterion = WeightedADHDLoss(weight_female_adhd=2)
# Calculate pos_weight based on class imbalance
sex_positive_weight = (len(y) - y['Sex_F'].sum()) / y['Sex_F'].sum()
sex_positive_weight = sex_positive_weight * 4
sex_criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(sex_positive_weight, dtype=torch.float32))

# Training loop
best_val_loss = np.inf
patience = 5
epochs_no_improve = 0

for epoch in range(50):
    model.train()
    running_loss = 0

    for xb, yb in train_loader:
        adhd_pred, sex_pred = model(xb)
        adhd_labels = yb[:, 0]
        sex_labels = yb[:, 1]

        loss_adhd = adhd_criterion(adhd_pred.squeeze(), adhd_labels, sex_labels)
        loss_sex = sex_criterion(sex_pred.squeeze(), sex_labels)
        loss = loss_adhd + loss_sex

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for xb, yb in test_loader:
            adhd_pred, sex_pred = model(xb)
            loss_adhd = adhd_criterion(adhd_pred.squeeze(), yb[:, 0], yb[:, 1])
            loss_sex = sex_criterion(sex_pred.squeeze(), yb[:, 1])
            val_loss = loss_adhd + loss_sex
            val_losses.append(val_loss.item())

    avg_val_loss = np.mean(val_losses)
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch+1} | Train Loss: {running_loss/len(train_loader):.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

# Evaluation
model.load_state_dict(best_model)
model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        adhd_pred, sex_pred = model(xb)
        adhd_pred = torch.sigmoid(adhd_pred)
        sex_pred = torch.sigmoid(sex_pred)
        preds = torch.cat([adhd_pred, sex_pred], dim=1)
        all_preds.append(preds)
        all_true.append(yb)

all_preds = torch.cat(all_preds).numpy()
all_true = torch.cat(all_true).numpy()

# Thresholding
adhd_pred_labels = (all_preds[:, 0] > 0.5).astype(int)
sex_pred_labels = (all_preds[:, 1] > 0.5).astype(int)

all_true_adhd = all_true[:, 0].astype(int)
all_true_sex = all_true[:, 1].astype(int)

# Metrics
def print_metrics(y_true, y_pred, name):
    print(f"\n{name} Metrics:")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")

print_metrics(all_true_adhd, adhd_pred_labels, "ADHD")
print_metrics(all_true_sex, sex_pred_labels, "Sex")

KeyError: "None of ['participant_id'] are in the columns"

In [60]:
X_select_test

,participant_id,fmri_pca_13,fmri_pca_2,ColorVision_CV_Score,fmri_pca_41,fmri_pca_46,MRI_Track_Scan_Location_CUNY,MRI_Track_Scan_Location_CBIC,APQ_P_APQ_P_CP,SDQ_SDQ_Prosocial,...,fmri_pca_31,fmri_pca_33,SDQ_SDQ_Emotional_Problems,fmri_pca_32,APQ_P_APQ_P_PP,Barratt_Barratt_P1_Edu_Graduate degree,"Barratt_Barratt_P2_Occ_Homemaker, stay at home parent.",fmri_pca_22,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Externalizing
0,Cfwaf5FX7jWK,-1.885215,0.798518,14.0,-1.742198,0.537296,1,0,5.0,8.0,...,0.793547,-0.849166,3.0,0.227520,26.0,1,0,1.479896,2.0,9.0
1,vhGrzmvA3Hjq,3.968658,-2.606679,12.0,-1.084569,-3.291825,1,0,3.0,9.0,...,1.124243,2.003162,8.0,1.661870,28.0,1,0,-1.941712,2.0,5.0
2,ULliyEXjy4OV,-1.428745,0.156742,13.0,0.495617,1.211216,1,0,3.0,9.0,...,1.016285,0.301984,1.0,2.472538,25.0,1,0,-2.552891,1.0,6.0
3,LZfeAb1xMtql,-0.860696,-2.720716,13.0,1.671720,0.295043,0,1,3.0,6.0,...,0.764329,0.126045,4.0,-0.856699,27.0,1,0,-1.041396,4.0,10.0
4,EnFOUv0YK1RG,-3.413086,-1.047284,14.0,-0.550988,-2.343593,1,0,3.0,10.0,...,-2.821317,-0.912005,6.0,-2.390853,28.0,0,0,-2.152863,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,UadZfjdEg7eG,-2.328931,-5.570785,14.0,-0.347845,2.276957,1,0,3.0,7.0,...,-0.433611,0.184916,7.0,-2.200096,25.0,0,0,0.342065,1.0,7.0
300,IUEHiLmQAqCi,-3.833227,-1.255951,14.0,1.773341,2.476305,0,1,5.0,8.0,...,0.383828,-4.780290,2.0,-0.680996,29.0,0,0,1.476167,2.0,11.0
301,cRySmCadYFRO,-3.497823,-3.607900,13.0,-0.178423,1.675037,1,0,3.0,10.0,...,2.515808,-0.729565,4.0,-1.360757,28.0,1,0,0.982247,1.0,4.0
302,E3MvDUtJadc5,3.256703,0.912379,14.0,0.667161,0.993344,1,0,3.0,0.0,...,3.339813,-2.718121,2.0,2.972512,22.0,1,0,2.026316,5.0,10.0


## Testing and Submission

In [63]:
# Standardize test data (using the same scaler from training)
# X_select_test = X_select_test.set_index('participant_id')
X_test_scaled = scaler.transform(X_select_test)

# Convert to PyTorch tensor
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Load test data into DataLoader (no need to include labels during testing)
test_loader_final = DataLoader(X_test_tensor, batch_size=32, shuffle=False)  # No shuffling during test

# Set model to eval mode and load best weights
model.load_state_dict(best_model)  # Load the best model state
model.eval()  # Set model to evaluation mode

adhd_preds, sex_preds = [], []

with torch.no_grad():
    for xb in test_loader_final:
        # Make predictions
        adhd_out, sex_out = model(xb)
        adhd_preds.append(adhd_out)
        sex_preds.append(sex_out)

# Concatenate predictions and apply threshold (0.5)
adhd_preds = torch.cat(adhd_preds).squeeze().numpy()  # ADHD predictions
sex_preds = torch.cat(sex_preds).squeeze().numpy()  # Sex predictions

# Ensure the lengths match
num_samples = len(X_select_test)
print(f"Number of samples: {num_samples}")
print(f"Predicted samples length: {len(adhd_preds)}")

# Threshold at 0.5 for binary classification
adhd_pred_labels = (adhd_preds > 0.5).astype(int)
sex_pred_labels = (sex_preds > 0.5).astype(int)

# Create test solution DataFrame
test_soln = pd.DataFrame({
    'participant_id': X_select_test.index,  # Assuming 'participant_id' is the index
    'ADHD_Outcome': adhd_pred_labels,
    'Sex_F': sex_pred_labels
})

# Save the DataFrame to a CSV for submission
test_soln.to_csv("submission.csv", index=False)

# Output the final solution for review
test_soln

Number of samples: 304
Predicted samples length: 304


,participant_id,ADHD_Outcome,Sex_F
0,Cfwaf5FX7jWK,1,0
1,vhGrzmvA3Hjq,1,1
2,ULliyEXjy4OV,1,1
3,LZfeAb1xMtql,1,1
4,EnFOUv0YK1RG,1,1
...,...,...,...
299,UadZfjdEg7eG,1,1
300,IUEHiLmQAqCi,1,1
301,cRySmCadYFRO,1,1
302,E3MvDUtJadc5,1,0
